In [1]:
x=1

In [2]:
# run in env scglue_new_conda
import networkx as nx
import scanpy as sc
import scglue
import pandas as pd

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/scglue_new_conda/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/scglue_new_conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
scglue.__version__

'0.4.0'

In [4]:
import subprocess
subprocess.run(["which", "bedtools"], check=True)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/scglue_new_conda/bin/bedtools


CompletedProcess(args=['which', 'bedtools'], returncode=0)

In [5]:
from scglue.check import check_deps
check_deps("bedtools")

In [6]:
import pybedtools
pybedtools.helpers.get_bedtools_path()

''

In [7]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/trimodal/data/raw/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad')
adata

AnnData object with n_obs × n_vars = 69249 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [8]:
atac = adata[:, adata.var['feature_types'] == 'ATAC'].copy()
rna = adata[:, adata.var['feature_types'] == 'GEX'].copy()
del adata

In [9]:
scglue.data.get_gene_annotation(
    rna, gtf = '/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/gencode.v49.annotation.gtf.gz',
    gtf_by="gene_name"
)
rna.var.loc[:, ["chrom", "chromStart", "chromEnd"]].head()

,chrom,chromStart,chromEnd
AL627309.5,NaN,NaN,NaN
LINC01409,chr1,778738.0,810066.0
LINC01128,chr1,825137.0,868835.0
NOC2L,chr1,943526.0,960714.0
KLHL17,chr1,960575.0,965719.0


In [10]:
rna = rna[:, ~rna.var['chrom'].isnull()].copy()
rna.var['chromStart'] = rna.var['chromStart'].astype(int)
rna.var['chromEnd'] = rna.var['chromEnd'].astype(int)
rna

AnnData object with n_obs × n_vars = 69249 × 12026
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_type', 'tag', 'hgnc_id', 'havana_gene', 'artif_dupl'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [11]:
hvf_mod1 = pd.read_csv('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/hvf_mod1.txt', header=None)
hvf_mod2 = pd.read_csv('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/hvf_mod2.txt', header=None)

hvf_mod1 = hvf_mod1[0].astype(str).tolist()
hvf_mod2 = hvf_mod2[0].astype(str).tolist()

hvf_mod1 = list(set(hvf_mod1) & set(rna.var_names))

In [12]:
rna.layers["counts"] = rna.X.copy()
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
sc.pp.scale(rna)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/scglue_new_conda/lib/python3.10/functools.py:889: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)


In [13]:
rna_hvg = rna[:, hvf_mod1].copy()
rna_hvg

AnnData object with n_obs × n_vars = 69249 × 1676
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_type', 'tag', 'hgnc_id', 'havana_gene', 'artif_dupl', 'mean', 'std'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism', 'log1p'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [14]:
split = atac.var_names.str.split(r"[:-]")
atac.var["chrom"] = split.map(lambda x: x[0])
atac.var["chromStart"] = split.map(lambda x: x[1]).astype(int)
atac.var["chromEnd"] = split.map(lambda x: x[2]).astype(int)

In [16]:
rna.var['highly_variable'] = [item in hvf_mod1 for item in rna.var_names]
rna.var['highly_variable'].sum()

np.int64(1676)

In [17]:
guidance = scglue.genomics.rna_anchored_guidance_graph(rna, atac)
guidance

window_graph: 100%|██████████| 12026/12026 [00:03<00:00, 3781.17it/s]


In [18]:
scglue.graph.check_graph(guidance, [rna, atac])

[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...


[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!


In [19]:
hvg_reachable = scglue.graph.reachable_vertices(guidance, rna_hvg.var_names)
hvg_reachable

{'PDCD10',
 'chr10-50421011-50421919',
 'chr9-21384922-21385799',
 'chr3-60080252-60081048',
 'chr3-111566691-111567514',
 'chr18-68735420-68736238',
 'chr10-89235107-89235991',
 'SLC16A1',
 'chr2-43152737-43153587',
 'IGLC3',
 'CD72',
 'chr8-8783312-8784227',
 'chr3-125055617-125056509',
 'chr10-60708399-60709269',
 'chr6-148867687-148868476',
 'chr5-124658998-124659871',
 'chr9-136237673-136238514',
 'NELL2',
 'chr13-99617264-99618148',
 'chr16-174694-175573',
 'chr9-113565313-113565981',
 'chr2-174989912-174990770',
 'chr20-60100743-60101593',
 'chr2-201269914-201270823',
 'chr17-66799758-66800639',
 'TENT5C',
 'chr22-25060353-25061286',
 'chr7-132575241-132576109',
 'chr1-206082960-206083808',
 'chr3-10238165-10239005',
 'chr2-191112306-191113204',
 'chr12-117069997-117070852',
 'chr4-80797999-80798830',
 'chr5-102764406-102765135',
 'chr14-34227752-34228637',
 'chr1-192810179-192810843',
 'chr12-6465271-6466139',
 'chr14-92475459-92476354',
 'chr9-95131100-95131982',
 'chr14-99260

In [20]:
graph_reachable = [item in hvg_reachable for item in atac.var_names]
hvp = list(set(atac.var_names[graph_reachable]) & set(hvf_mod2))

atac.var['highly_variable'] = [item in hvg_reachable for item in atac.var_names]
atac.var['highly_variable'].sum()

np.int64(16205)

In [21]:
subgraph = guidance.subgraph(hvg_reachable)

In [22]:
print(guidance)

MultiDiGraph with 128516 nodes and 271752 edges


In [23]:
print(subgraph)

MultiDiGraph with 18293 nodes and 53067 edges


In [24]:
nx.write_graphml(subgraph, '../pipeline/data/input/multiome/guidance-hvf.graphml.gz')